In [2]:
# %pip install --upgrade --quiet langchain langchain-community chromadb bs4 boto3
# %pip install --upgrade --quiet pydantic
# %pip install --upgrade --quiet sentence-transformers

# %pip install --upgrade --quiet sagemaker

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [24]:
# %pip install --upgrade --quiet trulens_eval
# %pip install --upgrade --quiet jinja2

# %pip install --upgrade --quiet evaluate
# %pip install --upgrade --quiet rouge_score
# %pip install --upgrade --quiet bert_score
%pip install --upgrade --quiet rouge-chinese

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install --upgrade ipywidgets
# "ipython>=8.12.0"

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip show trulens_eval

Name: trulens-eval
Version: 0.28.2
Summary: Library to systematically track and evaluate LLM based applications.
Home-page: https://www.trulens.org
Author: Truera Inc
Author-email: all@truera.com
License: MIT
Location: /home/ubuntu/workshops/.conda/lib/python3.11/site-packages
Requires: alembic, dill, frozendict, humanize, langchain, langchain-core, merkle-json, millify, munch, nest-asyncio, nltk, numpy, packaging, pip, psutil, pydantic, python-dotenv, requests, rich, sqlalchemy, streamlit, streamlit-aggrid, streamlit-extras, streamlit-pills, tqdm, typing-extensions, typing-inspect
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip show langchain

Name: langchain
Version: 0.1.16
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/ubuntu/workshops/.conda/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: trulens-eval
Note: you may need to restart the kernel to use updated packages.


In [1]:
from typing import Dict
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json

sagemaker_endpoint_name = 'chatglm3-6b-endpoint-2024-04-30-09-58-32-919'

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        parameters = model_kwargs
        if 'temperature' not in parameters:
            parameters['temperature'] = 0.01
            parameters['max_new_tokens'] = 512
        input = {"inputs": prompt, "parameters": parameters }
        input_str = json.dumps(input).encode('utf-8')
        return input_str
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["generated_text"]

chatglm_model = SagemakerEndpoint(
    endpoint_name=sagemaker_endpoint_name, 
    region_name="us-east-1", 
    content_handler=ContentHandler()
)


In [2]:
%%time
result = chatglm_model.invoke("Amazon Bedrock是什么?")
print(result)


Amazon Bedrock是一个由亚马逊开发的软件开发工具,旨在帮助开发人员构建和部署应用程序。Bedrock是一个基于后端API的构建系统,它使用现代的软件构建工具和语言,如.NET、Java和Python等,来构建和打包应用程序。

Bedrock的目标是提供一个可扩展、可靠和安全的构建系统,它具有以下特点:

1. 可扩展性:Bedrock使用云技术来提供可扩展的构建能力,可以在需要时自动扩展或缩小构建资源。

2. 可靠性:Bedrock使用自动化的工具和流程来确保构建过程的可靠性和一致性。

3. 安全性:Bedrock使用安全工具和流程来保护应用程序代码和数据的安全性。

Bedrock适用于需要构建和部署大型、复杂应用程序的开发团队。它可以帮助开发人员提高生产力、降低成本,并确保应用程序的可靠性和安全性。
CPU times: user 26 ms, sys: 0 ns, total: 26 ms
Wall time: 5.56 s


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='moka-ai/m3e-base')

In [4]:
import bs4
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Load, chunk and index the contents of the blog.

persist_directory="./chroma_500chunk/"
if (os.path.exists(persist_directory)):
    vectorstore = Chroma(embedding_function=embeddings, persist_directory=persist_directory)
else:
    loader = WebBaseLoader("https://huggingface.co/blog/zh/moe")
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    print('splits doc count:', len(splits))
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory=persist_directory)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [5]:
golden_set = [
    {'query': 'MOE架构的模型有什么优势?',    'response': '相对来说，MOE模型有更快的训练速度和推理速度，并且它不需要将所有参数都加载到显存中，所以需要较小的显存。'},
    {'query': '为什么MOE的训练和推理的速度更快?',    'response': '通过门控网络路由，每个token只选择一小部分专家网络计算，所以使用的显存较小，相对同等参数规模的模型，训练和推理速度也更快。而且通过并行不同的专家网络，可通过并行提高训练和推理效率。'},
    {'query': '简单总结MoE模型的特点?',    'response': '与相同参数量的模型相比，训练和推理都更快，但是在训练和调优时，都有些额外的挑战。'},
    {'query': 'Shazeer 将 MoE 应用在了哪个应用领域?',    'response': '主要集中在翻译领域。'},
    {'query': 'MoE模型的训练过程中，如何保证令牌不会总是被发送给少数几个受欢迎的专家??',    'response': '混合专家模型通过以下两种机制，保证令牌的负载均衡，首先引入一个辅助损失，确保所有专家接收到的训练样本数大致相等；其次，再引入专家容量，定义一个专家能处理的令牌的阈值'},
    {'query': 'MoE模型训练时，怎么保证token的负载均衡?',    'response': '混合专家模型通过以下两种机制，保证令牌的负载均衡，首先引入一个辅助损失，确保所有专家接收到的训练样本数大致相等；其次，再引入专家容量，定义一个专家能处理的令牌的阈值。'},
    {'query': 'GShard如何保证MoE模型的负载均衡?',    'response': 'GShard的作者，为了保证令牌的负载均衡，除了引入辅助损失，确保所有专家接收到的训练样本数大致相等，还通过随机路由确保top-2设置中的专家的权重；再通过设置专家容量，定义一个专家能处理的令牌的阈值。'},
    {'query': 'MoE的专家数量达到多少以后，效率就会显著降低?',    'response': '当专家数量达到 256 或 512 之后，效率的降低更为明显。'},
    {'query': '怎么能够把MOE模型变得更小，让它更适合部署?',    'response': '为了使模型更适合部署，有以下几种技术，预蒸馏的实验、任务界别的路由、以及专家网络聚合。'},
    {'query': '在微调MoE这样的稀疏模型时，怎么防止出现过拟合?',    'response': '可以使用使用辅助损失来避免过拟合，并设置较高的dropout 率，来优化模型性能。'},
    {'query': '模型的并行有几种形式?',    'response': '数据并行，模型并行，模型和数据并行，专家并行。'},
    {'query': '有哪些开源的MOE模型?',    'response': '有 Switch Transformers(Google)，NLLB MoE (Meta)，OpenMoE， Mixtral 8x7B (Mistral)'}
]


In [6]:
results = vectorstore.similarity_search(golden_set[0]['query'], k=2)
for r in results:
    print(r.page_content)

了解了 MoE 的基本概念后，让我们进一步探索推动这类模型发展的研究。





		混合专家模型简史
	

混合专家模型 (MoE) 的理念起源于 1991 年的论文 Adaptive Mixture of Local Experts。这个概念与集成学习方法相似，旨在为由多个单独网络组成的系统建立一个监管机制。在这种系统中，每个网络 (被称为“专家”) 处理训练样本的不同子集，专注于输入空间的特定区域。那么，如何选择哪个专家来处理特定的输入呢？这就是门控网络发挥作用的地方，它决定了分配给每个专家的权重。在训练过程中，这些专家和门控网络都同时接受训练，以优化它们的性能和决策能力。
在 2010 至 2015 年间，两个独立的研究领域为混合专家模型 (MoE) 的后续发展做出了显著贡献:
[Switch Transformers paper](https://arxiv.org/abs/2101.03961) 论文中的 MoE layer

总结来说，在混合专家模型 (MoE) 中，我们将传统 Transformer 模型中的每个前馈网络 (FFN) 层替换为 MoE 层，其中 MoE 层由两个核心部分组成: 一个门控网络和若干数量的专家。
尽管混合专家模型 (MoE) 提供了若干显著优势，例如更高效的预训练和与稠密模型相比更快的推理速度，但它们也伴随着一些挑战:


In [7]:
from trulens_eval import Tru

# initialize evaluation db
tru = Tru()
# tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [8]:
from trulens_eval.tru_custom_app import instrument
from langchain.prompts import PromptTemplate

prompt_template = """请基于以下的3引号里面的内容，简洁、准确的回答最后的问题，并使用中文。如果你无法从已知内容获取答案，就直接返回'根据已知信息无法回答该问题。'，不要编造答案。

已知内容:
```
{context}
```

问题: {question}
答案:"""

class RAG_App:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vectorstore.similarity_search(query, k=5)
        docs = [doc.page_content for doc in results]
        return docs

    @instrument
    def generate_completion(self, query: str, context: list) -> str:
        """
        Generate answer from context.
        """
        context_str = "\n\n".join(context)
        p = prompt_template.format(question=query, context=context_str)
        return chatglm_model.invoke(p)

    @instrument
    def query(self, query: str) -> str:
        context_slist = self.retrieve(query)
        completion = self.generate_completion(query, context_slist)
        return completion

rag = RAG_App()

In [9]:
from trulens_eval import Bedrock
from typing import Dict, Optional, Sequence
from trulens_eval.feedback import GroundTruthAgreement

class Claude3_Bedrock(Bedrock):
    def _create_chat_completion(
        self,
        prompt: Optional[str] = None,
        messages: Optional[Sequence[Dict]] = None,
        **kwargs
    ) -> str:
        assert self.endpoint is not None

        import json
        system_prompt = None
        if messages:
            if len(messages) == 1:
                messages[0]['role'] = 'user'
            elif messages[0]['role'] == 'system':
                system_prompt = messages[0]['content']
                messages.pop(0)
        elif prompt:
            messages = [{
                "role": "user",
                "content": prompt
            }]
        else:
            raise ValueError("Either 'messages' or 'prompt' must be supplied.")
        
        body = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 2048,
                "temperature": 0.01,
                "messages": messages
        }
        if system_prompt:
            body['system'] = system_prompt
        response = self.endpoint.client.invoke_model(body=json.dumps(body), modelId=self.model_id)
        response_body = json.loads(response.get('body').read()).get('content')[0]['text']
        return response_body
    

from typing import Dict, Tuple, Union
from trulens_eval.utils.generated import re_0_10_rating
import re, json

def extract_between_tags(tag: str, string: str) -> list[str]:
    ext_list = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    if ext_list:
        return ext_list[0]
    else:
        return ''

class RAGGroundTruthAgreement(GroundTruthAgreement):
    def agreement_measure_with_cot(
        self, prompt: str, response: str
    ) -> Union[float, Tuple[float, Dict[str, str]]]:
        ground_truth_response = self._find_response(prompt)

        AGREEMENT_SYSTEM_PROMPT = """你的任务是根据给定的提示词和参考答案,评价一个AI生成的答案。具体来说:

1. 首先,仔细阅读提示词,这是AI生成答案时需要遵守的要求：
<prompt>%(prompt)s</prompt>

2. 然后,查看参考答案，这是人类专家给出的理想答案：
<reference>%(reference)s</reference> 

3. 最后,评估AI生成的答案,看它是否满足了提示词的要求,以及与参考答案有多大出入：
<answer>%(answer)s</answer>

在评估时,首先以 ‘score:’开头,给出一个0-10分的分数,10分表示完全满足要求且与参考答案一致,0分表示完全不符合要求。
然后在 reason: 后写下你的评价理由和分析,解释AI生成答案的优缺点。

因此,你的回复应该按照如下格式:

score: 0-10分数值
reason:
...你对AI生成答案的评价理由...

请谨记,你的评分和理由应该客观公正,既不过于苛刻,也不过于宽松。对AI生成的答案进行全面而中肯的评估。
"""

        agreement_txt = self.provider.endpoint.run_in_pace(
            func=self.provider._create_chat_completion,
            prompt=(AGREEMENT_SYSTEM_PROMPT % {'prompt': prompt, 'reference': ground_truth_response, 'answer': response})
        )
        score = re_0_10_rating(agreement_txt)
        reason_index = agreement_txt.find('reason:')
        if reason_index < 0:
            reason_index = 0
        reason = agreement_txt[reason_index:]
        ret = score / 10, dict(
            ground_truth_response=ground_truth_response,
            reason=reason
        )
        return ret
    


In [10]:
fclaude = Claude3_Bedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", region_name='us-east-1')

In [11]:
from trulens_eval import Feedback, Select, Bedrock

from trulens_eval.feedback import Groundedness, GroundTruthAgreement
import numpy as np

grounded = Groundedness(groundedness_provider=fclaude)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(fclaude.relevance_with_cot_reasons, name = "Answer Relevance").on_input_output()

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fclaude.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

# Create a Feedback object for ground truth similarity
old_ground_truth = GroundTruthAgreement(golden_set, provider = fclaude)
f_old_groundtruth = (Feedback(old_ground_truth.agreement_measure, name = "Ground Truth")
                 .on_input_output()
                )

# custom ground truth 
ground_truth = RAGGroundTruthAgreement(golden_set, provider = fclaude)
# Call the agreement measure on the instruction and output
f_groundtruth = (Feedback(ground_truth.agreement_measure_with_cot, name = "Ground Truth")
                #  .on(Select.Record.calls[0].args.args[0])
                 .on_input_output()
                )
# f_bert = Feedback(ground_truth.bert_score).on_input_output()
# f_bleu = Feedback(ground_truth.bleu).on_input_output()
# f_rouge = Feedback(ground_truth.rouge).on_input_output()

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets.collect() .
✅ In Ground Truth, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Ground Truth, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Ground Truth, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Ground Truth, input response will be set to __record__.main_output or `Select.RecordOutput` .


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
from trulens_eval import TruCustomApp

tru_rag_old = TruCustomApp(rag, app_id='ChatGLM_rag_old', feedbacks=[f_groundedness, f_old_groundtruth, f_qa_relevance, f_context_relevance])

# with custom ground truth
# tru_rag = TruCustomApp(rag, app_id='ChatGLM_rag', feedbacks=[f_groundedness, f_groundtruth, f_qa_relevance, f_context_relevance])

# tru_rag_full = TruCustomApp(rag, app_id='ChatGLM_500Chunk_full', feedbacks=[f_groundtruth, f_bert, f_bleu, f_rouge])


In [13]:
with tru_rag_old as recording:
    rag.query(golden_set[8]['query'])

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
with tru_rag_old as recording:
    for question in golden_set:
        rag.query(question['query'])

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
tru.get_leaderboard(app_ids=[])

,Ground Truth,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,,
ChatGLM_extract5,0.868836,NaN,NaN,NaN,3.000000,0.0
ChatGLM_extract4,0.866071,NaN,NaN,NaN,3.000000,0.0
ChatGLM_extract6,0.859321,NaN,NaN,NaN,3.000000,0.0
ChatGLM_extract,0.825000,NaN,NaN,NaN,3.000000,0.0
ChatGLM_rag_old,0.800000,0.878571,0.778571,1.0,2.785714,0.0
ChatGLM_extract3,0.797583,NaN,NaN,NaN,3.000000,0.0
ChatGLM_rag,0.750000,0.866667,0.775000,1.0,3.083333,0.0
ChatGLM_extract2,0.727786,NaN,NaN,NaN,3.000000,0.0
ChatGLM_extract1,NaN,NaN,NaN,NaN,3.000000,0.0


In [ ]:
# with tru_recorder as recording:
#     for question in questions:
#         rag_chain.invoke(question)

In [17]:
tru.run_dashboard()
# tru.stop_dashboard() # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

: 

: 

In [ ]:
tru.reset_database()

### 要点总结和信息提取等场景

In [71]:
golden_set_ext = [
    {
        "query": "2023年8月3日，韩国京畿道城南市，22岁嫌疑人崔某驾驶汽车撞人后进入皇后百货商场持刀行凶，导致1人死亡，13人受伤。",
        "response": "时间: ['2023年8月3日']\n地点: ['韩国京畿道城南市']\n人名: ['崔某']"
    },
    {
        "query": "2023年8月11日，中国第13次北冰洋科学考察的第一阶段任务在中北冰洋太平洋扇区完成，考察队领队王金辉宣布，完成了大气、海洋、生物群落和资源调查等多项作业项目。",
        "response": "时间: ['2023年8月11日']\n地点: ['中北冰洋太平洋扇区']\n人名: ['王金辉']"
    },
    {
        "query": "2023年7月20日，国家防总副总指挥、水利部部长李国英表示，海河流域北三河地区将出现暴雨洪水过程，水利部积极部署相关防御工作。",
        "response": "时间: ['2023年7月20日']\n地点: ['海河流域北三河地区']\n人名: ['李国英']"
    },
    {
        "query": "2023年8月8日，京津冀、东北等地受到“杜苏芮”台风引发的严重洪涝灾害，圆通速递公司董事长喻会蛟宣布捐赠1000万元并发挥快递网络和供应链优势进行救援。",
        "response": "时间: ['2023年8月8日']\n地点: ['京津冀、东北']\n公司: ['圆通速递\n人名: ['喻会蛟']"
    }
]

In [66]:
from trulens_eval import Feedback, Select, Bedrock

from trulens_eval.feedback import Groundedness, GroundTruthAgreement
import numpy as np

class ItemizedGroundTruthAgreement(GroundTruthAgreement):
    def agreement_measure_with_cot(
        self, prompt: str, response: str
    ) -> Union[float, Tuple[float, Dict[str, str]]]:
        ground_truth_response = self._find_response(prompt)

        AGREEMENT_SYSTEM_PROMPT = """你的任务是对参考答案和生成答案进行比较,并计算准确率、召回率和F值。

这是参考答案:
<expected_answer>
%(reference)s
</expected_answer>

这是生成答案:
<generated_answer>
%(answer)s
</generated_answer>

首先,检查参考答案和生成答案的内容，得到两个集合。然后计算以下指标:

准确率 = 生成答案中正确词数 / 生成答案总词数
召回率 = 生成答案中正确词数 / 参考答案总词数
F值 = 2 * (准确率 * 召回率) / (准确率 + 召回率)

请在<reasoning>标签中写下你的计算过程,然后用JSON格式输出准确率、召回率和F值,格式如下:

<reasoning>
...你的计算过程...
</reasoning>

<result>
{
"accuracy": 准确率值,
"recall": 召回率值,
"f_score": F值
}
</result>
"""
        agreement_txt = self.provider.endpoint.run_in_pace(
            func=self.provider._create_chat_completion,
            prompt=(AGREEMENT_SYSTEM_PROMPT % {'reference': ground_truth_response, 'answer': response})
        )
        scores = extract_between_tags('result', agreement_txt)
        scores = json.loads(scores)
        
        score = scores['f_score']
        reason = extract_between_tags('reasoning', agreement_txt)
        ret = score, dict(
            ground_truth_response=ground_truth_response,
            scores=scores,
            reason=reason
        )
        return ret

In [72]:
# Create a Feedback object for ground truth similarity
itemized_ground_truth = ItemizedGroundTruthAgreement(golden_set_ext, provider = fclaude)
f_itemized_groundtruth = (Feedback(itemized_ground_truth.agreement_measure_with_cot, name = "Ground Truth")
                          .on_input_output()
                          )

from trulens_eval.tru_custom_app import instrument

ext_prompt_template = """从下面的上下文中提取出时间、地点、公司、人名等信息。

上下文:
```
{context}
```

答案按下面等格式返回：
时间: ["提取的时间"]
地点: ["提取的地点"]
公司: ["提取的公司名"]
人名: ["提取的人名"]

如果没有人名、公司名等信息，这一项就返回空列表：
时间: ["提取的时间"]
地点: ["提取的地点"]
公司: []
人名: []
"""

class Extract_App:

    @instrument
    def extract(self, context: str) -> str:
        p = ext_prompt_template.format(context=context)
        return chatglm_model.invoke(p)

extract = Extract_App()

from trulens_eval import TruCustomApp
tru_extract = TruCustomApp(extract, app_id='ChatGLM_extract6', feedbacks=[f_itemized_groundtruth])

Function <function Extract_App.extract at 0xed418c5c28e0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.Extract_App object at 0xed418c217050> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function Extract_App.extract at 0xed418c31c180> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.Extract_App object at 0xed418c217050> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function Extract_App.extract at 0xed418c31da80> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.Extract_App object at 0xed418c217050> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function Extract_App.extract at 0xed418c31e340> was not found during instrumentation walk. Make sure it is accessible by tr

✅ In Ground Truth, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Ground Truth, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [61]:
for qq in golden_set_ext:
    print(qq['query'])
    print(extract.extract(qq['query']))

2023年8月3日，韩国京畿道城南市，22岁嫌疑人崔某驾驶汽车撞人后进入皇后百货商场持刀行凶，导致1人死亡，13人受伤。



时间: ["2023年8月3日"]
地点: ["韩国京畿道城南市"]
公司: []
人名: ["崔某"]
2023年8月11日，中国第13次北冰洋科学考察的第一阶段任务在中北冰洋太平洋扇区完成，考察队领队王金辉宣布，完成了大气、海洋、生物群落和资源调查等多项作业项目。

时间: 2023年8月11日
地点: 中北冰洋太平洋扇区
公司: 无
人名: 王金辉
2023年7月20日，国家防总副总指挥、水利部部长李国英表示，海河流域北三河地区将出现暴雨洪水过程，水利部积极部署相关防御工作。

时间: ["2023年7月20日"]
地点: ["海河流域北三河地区"]
公司: ["水利部"]
人名: ["李国英"]
2023年8月8日，京津冀、东北等地受到“杜苏芮”台风引发的严重洪涝灾害，圆通速递公司董事长喻会蛟宣布捐赠1000万元并发挥快递网络和供应链优势进行救援。

时间: 2023年8月8日
地点: 京津冀、东北等地
公司: 圆通速递公司
人名: 喻会蛟


In [73]:

with tru_extract as recording:
    for question in golden_set_ext:
        extract.extract(question['query'])

In [74]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Ground Truth,Answer Relevance,latency,total_cost
app_id,,,,,,
ChatGLM_rag,0.775,1.0,0.750000,0.866667,3.083333,0.0
ChatGLM_extract5,NaN,NaN,0.868836,NaN,3.000000,0.0
ChatGLM_extract4,NaN,NaN,0.866071,NaN,3.000000,0.0
ChatGLM_extract6,NaN,NaN,0.859321,NaN,3.000000,0.0
ChatGLM_extract,NaN,NaN,0.825000,NaN,3.000000,0.0
ChatGLM_extract3,NaN,NaN,0.797583,NaN,3.000000,0.0
ChatGLM_extract2,NaN,NaN,0.727786,NaN,3.000000,0.0
ChatGLM_extract1,NaN,NaN,NaN,NaN,3.000000,0.0


In [5]:
from langchain_community.chat_models import BedrockChat
import boto3

session = boto3.Session()
bedrock_client = session.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'
)
model_kwargs = {
    "max_tokens": 2048,
    "temperature": 0.01,
    # "top_k": 250,
    # "top_p": 1,
}

claude3_llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client,
    model_kwargs=model_kwargs
)

/home/ubuntu/workshops/.conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import ChatBedrock`.
  warn_deprecated(


In [28]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    HumanMessage(content="星际穿越这部电影怎么样?")
]
claude3_llm(messages)

/home/ubuntu/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='星际穿越是一部非常出色的科幻电影,从多个方面都值得一看:\n\n1. 科学性强 - 电影中涉及到的相对论、黑洞、高维度等概念都有一定的科学依据,导演与著名物理学家科普作家合作,努力让科学内容尽可能贴近现有理论。\n\n2. 视觉效果震撼 - 太空旅行、异次元空间等场景的视觉呈现非常梦幻而宏大,给人深刻的视觉冲击。\n\n3. 剧情设置巧妙 - 人类面临资源枯竭的未来,科学家们孤注一掷寻找新家园的故事情节紧凑且富有张力。\n\n4. 探讨哲理命题 - 影片蕴含了对爱、家庭、牺牲、人性等命题的思考,给人以启迪。\n\n5. 演员阵容出色 - 马修·麦康纳、安妮·海瑟薇等主演都有精彩的演绎。\n\n总的来说,这部电影在视觉体验和思想内涵上都达到了很高的水准,是一部值得反复观赏的佳作。你看过这部影片吗?有何其他感想?', response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 22, 'completion_tokens': 396, 'total_tokens': 418}}, id='run-8dbbbbe1-955d-4779-bcdc-4ecc687d0311-0')

In [30]:
# 与上面的调用一样，后面会使用message API调用
claude3_llm.invoke("星际穿越这部电影怎么样?")

AIMessage(content='星际穿越是一部非常出色的科幻电影,从多个方面都值得一看:\n\n1. 科学性强 - 电影中涉及到的相对论、黑洞、高维度等概念都有一定的科学依据,导演与著名物理学家科普作家合作,努力让科学内容尽可能贴近现有理论。\n\n2. 视觉效果震撼 - 太空旅行、异次元空间等场景的视觉呈现非常梦幻而宏大,给人深刻的视觉冲击。\n\n3. 剧情设置巧妙 - 人类为了生存不得不去寻找新家园的故事情节引人入胜,加上亲情的元素也很打动人心。\n\n4. 哲理性强 - 影片蕴含了对时间、爱、生命等哲学命题的思考,给人以启迪。\n\n5. 演员阵容强大 - 马修·麦康纳、安妮·海瑟薇等主演都演技出众。\n\n总的来说,这部电影在娱乐性与思考性之间取得了很好的平衡,是一部值得细细品味的佳作。你看过这部影片吗?有何其他感想?', response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 22, 'completion_tokens': 381, 'total_tokens': 403}}, id='run-25cd6003-5bf3-4644-aa31-98bb0399394f-0')

In [7]:
eval_prompt = """你的任务是根据给定的提示词和参考答案,评价一个AI生成的答案。具体来说:

1. 首先,仔细阅读提示词，这是AI生成答案时需要遵守的要求：
<prompt>{$PROMPT}</prompt>

2. 然后,查看参考答案，这是人类专家给出的理想答案：
<reference>{$REFERENCE}</reference> 

3. 最后,评估AI生成的答案，看它是否满足了提示词的要求,以及与参考答案有多大出入：
<answer>{$ANSWER}</answer>

在评估时,首先以 ‘score:’开头，给出一个0-10分的分数,10分表示完全满足要求且与参考答案一致,0分表示完全不符合要求。
然后在 justification: 后写下你的评价理由和分析,解释AI生成答案的优缺点。

因此,你的回复应该按照如下格式:

score: 0-10分数值
justification:
...你对AI生成答案的评价理由...

请谨记,你的评分和理由应该客观公正,既不过于苛刻,也不过于宽松。对AI生成的答案进行全面而中肯的评估。
"""

In [8]:
ref = "谢谢您的提问! Bedrock可以利用知识库实现检索增强生成。"
answer = "Bedrock与知识库相结合，实现自动化RAG解决方案，加速上市速度，提高成本效益。"

prompt = """仅使用提供的上下文回答问题。如果不在上下文内容，就说你不知道，不要试图编造答案。答案尽量不要超过三个句子，并尽可能保持答案简洁。总是说“谢谢您的提问！”在答案的开头。
上下文:
借助 Amazon Bedrock，您可以使用知识库启用检索增强生成 (RAG) 工作流程，并利用 LLM 的推理功能构建上下文应用程序。
RAG 是一种流行的技术，它将私有数据的使用与大型语言模型 (LLM) 相结合。
Amazon Bedrock 与知识库相结合，通过自动化 RAG 解决方案并减少代理的构建时间，加快上市速度。 添加知识库还可以消除不断训练模型以利用私有数据的需要，从而提高成本效益
。RAG 首先根据用户的查询从数据存储（最常见的是向量索引）检索相关文档。 然后，它使用语言模型通过考虑检索到的文档和原始查询来生成响应。 知识库服务会自动为您执行以下 RAG 设置和实施步骤3

问题: 请简明扼要用30个汉字以内来回答Bedrock与知识库什么关系？"""

eval_prompt = eval_prompt.replace('{$REFERENCE}', ref).replace('{$ANSWER}', answer).replace('{$PROMPT}', prompt)

In [9]:
eval_result = claude3_llm.invoke(eval_prompt)
print(eval_result.content)

score: 8
justification: 
AI生成的答案总体上符合提示词的要求,并且与参考答案的内容基本一致。答案简洁地概括了Bedrock与知识库的关系,即Bedrock可以利用知识库实现自动化的检索增强生成(RAG)解决方案,从而加快上市速度和提高成本效益。这与参考答案"Bedrock可以利用知识库实现检索增强生成"的核心内容相符。

不过,AI生成答案略显冗长,未能完全遵循"尽量不要超过三个句子"的要求。此外,它也没有在开头说"谢谢您的提问!"。因此,虽然内容方面较为完整,但在遵循提示词的细节要求上还有一些欠缺,所以给予8分较为合理。

总的来说,AI生成的答案质量较高,能够准确回答问题,只是在一些细节方面与提示词要求有一些出入。


In [13]:
from langchain.globals import set_verbose
set_verbose(True)

In [32]:
rag.query("MOE有什么优势?")

'根据已知信息，混合专家模型（MoE）的优势包括：1. 更高效的预训练，2. 与稠密模型相比，推理速度更快。'

In [71]:
rag_chain.invoke("为什么MOE的训练和推理的速度更快?", )

<class 'langchain_core.documents.base.Document'>


'根据 Switch Transformers 论文中的描述，混合专家模型 (MoE) 通过将传统 Transformer 模型中的每个前馈网络 (FFN) 层替换为 MoE 层，其中 MoE 层由两个核心部分组成: 一个门控网络和若干数量的专家。这种替换可以提高训练和推理速度。具体来说，由于 MoE 模型只使用其中的一部分参数，所以在推理过程中只需要加载部分参数到内存中，从而降低了内存需求。此外，由于 MoE 模型中的专家数量相对较少，因此训练和推理过程中计算量更小，从而提高了速度。'

In [77]:
rag_chain.invoke("简单总结MoE模型的特点")

<class 'langchain_core.documents.base.Document'>


'MoE模型是一种混合专家模型的简写，它将传统Transformer模型中的每个前馈网络（FFN）层替换为MoE层，其中MoE层由两个核心部分组成：一个门控网络和若干数量的专家。MoE模型具有以下特点：1. 训练效率高，能够实现更高效的计算预训练；2. 与稠密模型相比，推理速度更快；3. 具有较好的泛化能力，但在微调阶段面临泛化能力不足的问题；4. 参数需求较大，推理过程中只使用其中一部分，需要将所有参数加载到内存中，因此对内存的需求较高。'

In [87]:
rag_chain.invoke("Shazeer将 MoE 应用在了哪个应用领域？")

<class 'langchain_core.documents.base.Document'>


'Shazeer 将 MoE 应用于自然语言处理（NLP）领域。'

In [91]:
rag_chain.invoke("MoE模型的训练过程中，如何保证令牌不会总是被发送给少数几个受欢迎的专家？")

'ST-MoE 论文中显示了哪些令牌组被发送给了哪个专家的表格。根据文中所述，混合专家模型 (MoE) 中的令牌会根据专家的权重比例被分配，以保证所有专家接收到大致相等数量的训练样本，从而平衡专家之间的选择。此外，辅助损失可以被用来鼓励给予所有专家相同的重要性，以避免令牌总是被发送给少数几个受欢迎的专家。'

In [92]:
rag_chain.invoke("MoE模型训练时，怎么保证token的负载均衡？")

'根据所提供的信息，MoE模型在训练时通过令牌路由和负载均衡机制来保证token的负载均衡。具体来说，该模型将令牌发送到拥有所需专家的节点，每个节点处理不同批次的训练样本。这种设计可以确保每个专家接收到不同数量的令牌，从而实现token的负载均衡。此外，该模型还采用了容量因子来提高模型性能，但这也意味着更高的通信成本和对保存激活值的显存的需求。在设备通信带宽有限的情况下，选择较小的容量因子可能是更佳的策略。'

In [93]:
rag_chain.invoke("GShard如何保证MoE模型的负载均衡？")

'GShard 保证 MoE 模型的负载均衡主要通过以下几个方面：\n1. 随机路由：在 Top-2 设置中，我们始终选择排名最高的专家，但第二个专家是根据其权重比例随机选择的。\n2. 专家容量：可以设定一个阈值，定义一个专家能处理多少令牌。如果两个专家的容量都达到上限，令牌就会溢出，并通过残差连接传递到下一层，或在某些情况下被完全丢弃。\n3. 辅助损失：为了鼓励给予所有专家相同的重要性，引入了一个辅助损失，确保所有专家接收到大致相等数量的训练样本，从而平衡了专家之间的选择。\n通过这些措施，GShard 能够实现 MoE 模型的负载均衡，从而提高训练效率。'

In [97]:
rag_chain.invoke("MoE的专家数量达到多少以后，效率就会显著降低？")

'根据所提供的信息，混合专家模型（MoE）的专家数量达到一定程度后，效率会显著降低。具体来说，当专家数量超过一定程度（约为1000-2000）时，训练和推理速度会明显变慢。因此，在选择专家数量时，需要权衡模型性能和计算资源的使用。'

In [99]:
chatglm_model("MoE的专家数量达到多少以后，效率就会显著降低？")

'这是一个比较复杂的问题，因为机器学习模型的效率受到很多因素的影响，包括数据集大小、模型结构、超参数选择等等。同时，专家数量也不是一个固定的值，而是需要根据具体问题进行调整。\n\n一般来说，当专家数量达到一定程度时，模型的效率可能会出现下降趋势。这是因为随着专家数量的增加，模型的复杂度也会增加，导致模型训练时间增加，同时过拟合的风险也会增加。但是，具体多少数量的专家会使得效率显著降低，需要根据具体问题和数据集情况进行实验和分析。\n\n另外，还有一些技巧可以帮助提高模型效率，例如正则化、早停、Dropout等。这些技巧可以在一定程度上减少过拟合现象，提高模型的泛化能力，同时也可以降低模型的复杂度，提高训练效率。'

In [100]:
rag_chain.invoke("在微调MoE这样的稀疏模型时，怎么防止出现过拟合？")

'根据ST-MoE论文，微调稀疏混合专家模型（MoE）时，可以通过以下方法防止过拟合：\n\n1. 降低学习率和调大批量可以提升稀疏模型微调质量。\n2. 尝试冻结所有非专家层的权重，实践中，这会导致性能大幅下降，但这符合预期，因为混合专家模型（MoE）层占据了网络的主要部分。可以尝试相反的方法：仅冻结MoE层的参数。实验结果显示，这种方法几乎与更新所有参数的效果相当。这种做法可以加速微调过程，并降低显存需求。\n3. 考虑特别的微调超参数设置，例如，稀疏模型往往更适合使用较小的批量大小和较高的学习率，这样可以获得更好的训练效果。'

In [102]:
rag_chain.invoke("怎么将MoE模型的推理阶段更加高效？")

'将MoE模型的推理阶段更加高效的方法包括：\n\n1. 优化门控网络和专家的权重训练过程，以提高模型在微调阶段的泛化能力。\n2. 采用更高效的计算方法，例如将稀疏混合专家模型 (SMoE) 蒸馏回具有更少实际参数但相似等价参数量的稠密模型。\n3. 探索合并专家模型的技术及其对推理时间的影响。\n4. 尝试对Mix'

In [103]:
rag_chain.invoke("模型的并行有几种形式？")

'问题: 模型的并行有几种形式？\n答案: 数据并行、模型并行、模型和数据并行、专家并行。'

In [104]:
rag_chain.invoke("部署 MoE 模型有哪些技术?")

'部署 MoE 模型有以下几种技术：\n\n1. 预先蒸馏 (Pre-distillation)：将 MoE 模型蒸馏回其对应的稠密模型，以保留稀疏性带来的性能提升，同时使得在推理中使用更小型的模型成为可能。\n\n2. 任务级别路由 (Task-level routing)：将整个句子或任务直接路由到一个专家，以提取出一个用于服务的子网络，有助于简化模型的结构。\n\n3. 专家网络聚合 (Expert network aggregation)：通过合并各个专家的权重，在推理时减少了所需的参数数量，在不显著牺牲性能的情况下降低模型的复杂度。'